# Import Data

In [ ]:
import tensorflow.keras as K
import numpy as np
from tqdm import tqdm
import pickle
from time import time

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as bk

from collections import defaultdict
#import spacy
from tqdm import tqdm_notebook as tqdm
import random

import pandas as pd
import altair as alt

from sklearn.neighbors import NearestNeighbors

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
"""from google.colab import auth
auth.authenticate_user()
"""

'from google.colab import auth\nauth.authenticate_user()\n'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
############# try sentences Dont break into words.... (Sentence UNIVE)

#### try the obj det encoder and try

In [ ]:
import pickle

with open('/content/drive/MyDrive/VisualGenemo/COMBINED_F_VG_AVG_obj_bbox_IMG_img_vgg19_full.pkl', 'rb') as handle:
  i_e=pickle.load(handle)

In [ ]:
import pickle

with open('/content/drive/MyDrive/VisualGenemo/MOD_COMBINED_F_VG_COCO_AVG_obj_bbox_IMG_img_vgg19_full.pkl', 'rb') as handle:
  i_e=pickle.load(handle)

In [ ]:
import pickle

with open('/content/drive/MyDrive/VisualGenemo/Combined_ALPHA_SVD_512_obj_COLL_Smoothing_alpha_SVD_NORM.pkl', 'rb') as handle:
    t_e=pickle.load(handle)


In [ ]:
im_e={}
tx_e={}
for i in list(i_e.keys()):
  if i in t_e:
    im_e[i]=i_e[i]
    tx_e[i]=t_e[i]
len(im_e)

17158

# Triplet Loss

In [ ]:
def triplet_loss_image_to_text(y_true, y_pred):
    """
    Computes the unidirectional triplet loss with (anchor_image, positive_text, negative_text)

    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)

    Output:
    - triplet_loss:a scalar, so no shape

    """
    margin = 0.005

    #delete? embeddings = K.reshape(y_pred, (-1, 3, output_dim))
    anchor_image = y_pred[0]
    positive_text = y_pred[1]
    anchor_image_ignore = y_pred[2]
    negative_text = y_pred[3]

    positive_distance = K.backend.mean(K.backend.square(anchor_image - positive_text),axis=-1)
    negative_distance = K.backend.mean(K.backend.square(anchor_image - negative_text),axis=-1)

    triplet_loss = K.backend.mean(K.backend.maximum(0.0, positive_distance - negative_distance + margin))

    print("triplet_loss = ",triplet_loss)

    return triplet_loss

In [ ]:
def triplet_loss_text_to_image_prime(y_pred):
    """
    Computes the unidirectional triplet loss with (anchor_image, positive_text, negative_text)

    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)

    Output:
    - triplet_loss:a scalar, so no shape

    """
    margin = 0.005

    positive_image = y_pred[0]
    anchor_text = y_pred[1]
    negative_image = y_pred[2]
    anchor_text_ignore = y_pred[3]


    positive_distance = K.backend.mean(K.backend.square(anchor_text - positive_image),axis=-1)
    negative_distance = K.backend.mean(K.backend.square(anchor_text - negative_image),axis=-1)

    triplet_loss = K.backend.mean(K.backend.maximum(0.0, positive_distance - negative_distance + margin))

    return triplet_loss

In [ ]:
def bidirectional_triplet_loss_prime(y_pred):
    """
    Computes the bidirectional triplet loss with (anchor_image, positive_text, negative_text)

    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)

    Output:
    - triplet_loss:a scalar, so no shape
    """

    #hyperparameter to weight the ratio of direction of loss
    lambda_1 = 0.01

    bidirectional_triplet_loss = triplet_loss_image_to_text_prime(y_pred) + lambda_1 * triplet_loss_text_to_image_prime(y_pred)

    return bidirectional_triplet_loss

In [ ]:
def triplet_loss_image_to_text_prime(y_pred):
    """
    Computes the unidirectional triplet loss with (anchor_image, positive_text, negative_text)

    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)

    Output:
    - triplet_loss:a scalar, so no shape

    """
    margin = 0.1

    #delete? embeddings = K.reshape(y_pred, (-1, 3, output_dim))
    anchor_image = y_pred[0]
    positive_text = y_pred[1]
    anchor_image_ignore = y_pred[2]
    negative_text = y_pred[3]

    positive_distance = K.backend.mean(K.backend.square(anchor_image - positive_text),axis=-1)
    negative_distance = K.backend.mean(K.backend.square(anchor_image - negative_text),axis=-1)

    triplet_loss = K.backend.sum(K.backend.maximum(0.0, positive_distance - negative_distance + margin))

    print("triplet_loss = ",triplet_loss)

    return triplet_loss

In [ ]:
def identity_loss(y_true, y_pred):
    return K.backend.mean(y_pred - 0 * y_true)

# Network

In [ ]:
#image network
input_image = Input(shape=(4096,),name='input_image')
x = Dense(4096)(input_image)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)# 256 -> 512
x = BatchNormalization()(x)
prediction_image = Lambda(lambda  x: bk.l2_normalize(x,axis=1))(x)

#text network
input_text = Input(shape=(512,),name='input_text')
#y = Dense(4096)(input_text)
#y = Activation('relu')(y)
#y = Dropout(0.5)(y)
#y = Dense(256)(y)
#y = BatchNormalization()(y)
prediction_text = input_text   # Lambda(lambda  x: bk.l2_normalize(x, axis=1))(y)


basic_model = Model(inputs=[input_image,input_text], outputs=[prediction_image,prediction_text])
#basic_model.compile(optimizer='rmsprop',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])
#print(basic_model.summary())

In [ ]:
#Shared model with multiple inputs
positive_image = Input(shape=(4096,))
positive_text = Input(shape=(512,))
negative_image = Input(shape=(4096,))
negative_text = Input(shape=(512,))
positive_basic_model = basic_model([positive_image,positive_text])
negative_basic_model = basic_model([negative_image,negative_text])

triplet_loss_image_to_text_PRIME = Lambda(triplet_loss_image_to_text_prime)([positive_basic_model[0], positive_basic_model[1],negative_basic_model[0],negative_basic_model[1]])

#merged_model = Model(inputs=[positive_image,positive_text,negative_image,negative_text], outputs=[positive_basic_model[0], positive_basic_model[1],negative_basic_model[0],negative_basic_model[1]])
merged_model = Model(inputs=[positive_image,positive_text,negative_image,negative_text], outputs=triplet_loss_image_to_text_PRIME)

ADAM = K.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#ADAM = K.optimizers.Adam()
merged_model.compile(optimizer=ADAM, loss=identity_loss, metrics=[])

#print(merged_model.summary())

triplet_loss =  Tensor("lambda_1/Sum:0", shape=(), dtype=float32)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Triplet Selection

In [ ]:
#generate labels

def triplet_generator(image_embeddings, text_embeddings, margin, basic_model, batch_size, option):
    """
    Generate triplets based on the "option" value.

    Inputs:
    - image_embeddings: A dictionary with values = embeddings of image, of shape (N, emb_image)
    - text_embeddings: A dictionary with values =  embeddings of text, of shape (N, emb_text)
    - margin: A scalar specifying margin
    - basic_model: A numpy array of weights, of shape (4, M)?
    - batch_size: A scalar specifying mini batch size
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives

    Output:
    - image_text_pair_keys: A list of randomly chosen image_text_pair_keys of size k.
        This might be useful for post processing later.
    - quadrapule:(positive_image_list, positive_text_list, negative_image_list, negative_txt_list)

    """
    #k hardest examples
    K=1

    #Create a list of randomly chosen image_text_pair_keys of size k
    #Example of an element in image_text_pair_keys = 31034578.png
    image_text_pair_keys = random.choices(list(text_embeddings.keys()),k=batch_size)

    #create a list of corresponding text_embeddings for image_text_pair
    text_embedding_of_image_text_pairs=list()
    image_embedding_of_image_text_pairs=list()
    for image_text_pair_key in image_text_pair_keys:
        text_embedding_of_image_text_pairs.append(random.choice(text_embeddings[image_text_pair_key])) #do you need random.choice here?
        image_embedding_of_image_text_pairs.append(random.choice(image_embeddings[image_text_pair_key])) #do you need random.choice here?
        #text_embedding_of_image_text_pairs.append(text_embeddings[image_text_pair_key])
        #image_embedding_of_image_text_pairs.append(image_embeddings[image_text_pair_key])



    #can we eliminate loops here?
    #positive_image_text_pair_embeddings is a list of size(2,batch_size, ????)
    #positive_image_text_pair_embedding = [[np.array(image_embedding_of_image_text_pairs[idx]).reshape(-1,4096),np.array(text_embedding_of_image_text_pairs[idx]).reshape(-1,512)] for idx in range(len(image_text_pair_keys))]
    positive_image_text_pair_embedding = [[np.array(image_embedding_of_image_text_pairs[idx]).reshape(-1,4096),np.array(text_embedding_of_image_text_pairs[idx]).reshape(-1,512)] for idx in range(len(image_text_pair_keys))]


    #image_embedding_minibatch = np.array(image_embedding_of_image_text_pairs)
    #text_embedding_minibatch = np.array(text_embedding_of_image_text_pairs)

    predictions = list()
    #positives_image = list()
    #positives_text = list()

    for inst in positive_image_text_pair_embedding:
        #print(inst[0].shape)
        #print(inst[1].shape)
        #print(type(inst))
        predictions.append(basic_model.predict(inst))

    #image_joint_embedding is numpy array of size(batch_size, N )
    image_joint_embedding = np.vstack([prediction[0] for prediction in predictions])
    #text_joint_embedding is numpy array of size(batch_size, N)
    text_joint_embedding = np.vstack([prediction[1] for prediction in predictions])

    #print(image_joint_embedding.shape)
    #print(text_joint_embedding.shape)

    if option in [0,1,5,6]:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_K_hardest_negatives
    elif option == 2 or option == 7:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_ALL_hard_negatives
    elif option == 3 or option == 8:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_ALL_semihard_negatives
    elif option == 4 or option == 9:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_ALL_hard_and_semihard_negatives

    positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch = generate_mini_batch_of_triplets(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option,batch_size,margin)

    return image_text_pair_keys, positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_ALL_hard_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of ALL hardest negatives.

    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss

    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
        Total number of triplets generates = cannot say for sure, but it will be between [batch_size,batch*size * 2]
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'

    K=1

    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1,metric = set_metric) #batch_size
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)

    #print(max(distance))
    #print(min(distance))

    positive_image_embedding_minibatch = np.empty(shape=[0, 4096])
    positive_text_embedding_minibatch = np.empty(shape=[0, 512])
    negative_image_embedding_minibatch = np.empty(shape=[0, 4096])
    negative_text_embedding_minibatch = np.empty(shape=[0, 512])

    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]

        #start from hardest negative and check if is hard negative or not?
        j = 0
        while (j < K+1) and (j != indices[i,j]):#batch_size
            np.append(positive_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(positive_text_embedding_minibatch, positive_image_text_pair_embedding[i][1],axis=0)
            np.append(negative_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(negative_text_embedding_minibatch, positive_image_text_pair_embedding[indices[i,j]][1],axis=0)
            j = j+1
    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_ALL_hard_and_semihard_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of ALL hard and semihard negatives.

    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss

    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
        Total number of triplets generates = cannot say for sure, but it will be between [batch_size,batch*size * 2]
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'

    K = 1

    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1,metric = set_metric)#batch_size
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)

    positive_to_anchor_distance = np.zeros((batch_size,))

    for idx in range(batch_size):
        for j in range(K+1):#batch_size
            if idx == indices[idx,j]:
                positive_to_anchor_distance[idx] = distances[idx,j]

    positive_image_embedding_minibatch = np.empty(shape=[0, 4096])
    positive_text_embedding_minibatch = np.empty(shape=[0, 512])
    negative_image_embedding_minibatch = np.empty(shape=[0, 4096])
    negative_text_embedding_minibatch = np.empty(shape=[0, 512])

    #positive_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    #positive_text_embedding_minibatch = np.zeros((batch_size*K,300))
    #negative_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    #negative_text_embedding_minibatch = np.zeros((batch_size*K,300))

    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]

        #start from hardest negative and check if is hard negative or not?
        j = 0
        while (j < K+1) and (distances[i,j] <= positive_to_anchor_distance[i]+margin):
            np.append(positive_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(positive_text_embedding_minibatch, positive_image_text_pair_embedding[i][1],axis=0)
            np.append(negative_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(negative_text_embedding_minibatch, positive_image_text_pair_embedding[indices[i,j]][1],axis=0)
            j = j+1
    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_ALL_semihard_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of ALL hardest negatives.

    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss

    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
        Total number of triplets generated = cannot say for sure, but it will be between [batch_size,batch*size * 2]
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'

    K = 1

    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1,metric = set_metric)#batch_size
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)

    positive_to_anchor_distance = np.zeros((batch_size,))

    for idx in range(batch_size):
        for j in range(K+1): #batch_size
            if idx == indices[idx,j]:
                positive_to_anchor_distance[idx] = distances[idx,j]

    positive_image_embedding_minibatch = np.empty(shape=[0, 4096])
    positive_text_embedding_minibatch = np.empty(shape=[0,512])
    negative_image_embedding_minibatch = np.empty(shape=[0, 4096])
    negative_text_embedding_minibatch = np.empty(shape=[0, 512])

    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]

        #start from hardest negative and check if is hard negative or not?
        j = 0
        while (j < K+1) and (distances[i,j] <= positive_to_anchor_distance[i]+margin):#batch_size
            if distances[i,j] > positive_to_anchor_distance[i]:
                np.append(positive_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
                np.append(positive_text_embedding_minibatch, positive_image_text_pair_embedding[i][1],axis=0)
                np.append(negative_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
                np.append(negative_text_embedding_minibatch, positive_image_text_pair_embedding[indices[i,j]][1],axis=0)
            j = j+1
    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_K_hardest_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of K hardest negatives.

    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss

    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
    Total number of triplets generated = batch_size * K
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'

    if option == 0:
        K=1
    elif option == 1:
        K=10

    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1, metric = set_metric)
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)

    #print("Max_distance = ",np.max(distances))
    #print("Min_distance = ",np.min(distances))

    positive_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    positive_text_embedding_minibatch = np.zeros((batch_size*K,512))
    negative_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    negative_text_embedding_minibatch = np.zeros((batch_size*K,512))

    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]

        #STATE is set to 1, when the positive text is found in the nearest K neighbours.
        # When STATE is set to 1, we just skip over that neighbouring point.
        STATE = 0

        #print("#"*20,i,"#"*20)
        #print("distance =",distances[i])
        #print("indices =",indices[i])
        #print("Min_distance = ",np.min(distances[i]))

        for j in range(0,K):
            positive_image_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[i][0]
            positive_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[i][1]
            negative_image_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[i][0]

            if i == indices[i,j]:
                STATE = 1
            #is this correct? Does this not mean that nearest neighbour is same as positive text?
            #shouldn't this be text_embedding_minibatch[indices[i,1]] instead???
                #negative_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embeddings[indices[i,j+state]][1]
            #else:
            #is this correct? Does this not mean that nearest neighbour is NOT same as positive text?
            #shouldn't this be text_embedding_minibatch[indices[i,0]] instead???
            #hard negative example
                #negative_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embeddings[indices[i,j+state]][1]
            negative_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[indices[i,j+STATE]][1]


    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

#Split Test and Train


In [ ]:
len(im_e),len(tx_e)

(24521, 24521)

In [ ]:
tx_e.keys()

dict_keys(['dapper', 'martial-artist', 'college/university', 'solemn', 'Truck', 'jeter', 'reddish-orange', 'festivity', 'peeson', 'royalty', 'pinata', 'counterpart', 'rotweiler', 'dumping', 'ring-bearer', 'thourgh', 'video-game', 'perform', 'injection', 'cadre', 'inspiron', 'phone..', 'forbidding', 'swaetshirt', 'altitude', 'relevance', 'pst', 'dag', 'supervision', 'meetinghall', 'flan', 'derelict', 'egss', 'bather', 'meseum', 'disrupt', 'thrift', 'pink-and-black', 'swing-set', 'overlook', 'staple', 'windsurfs', 'plethora', 'jewelery', 'tootbrush', 'wile', 'bannas', 'meatless', 'contraction', 'bathhouse', 'stoll', 'reviews', 'sombreros', 'malabarism', 'concern', 'rival', 'rawhide', 'serape', 'naan', 'trail..', 'headscarfs', 'surrounding', 'bash', 'igloo', 'bartending', 'plumber', '.racquet', 'dread', 'pudgy', 'nip', 'personality', 'tiday', 'forge', 'zeppelin', 'fileed', 'grandma', 'quaint', 'messing', 'thesurface', 'ccourt', 'sofe', 'caliber', 'sunset..', 'supervise', 'os', 'peek', 'di

In [ ]:
im_e.pop("year'")
tx_e.pop("year'")

In [ ]:
len(im_e),len(tx_e)

(24520, 24520)

In [ ]:
c=0
e=list(im_e.keys())
t=[]
for j in range(100):
  c=0
  e=list(im_e.keys())
  for i in list(im_e.values()):
    if np.any(np.isnan(i)):
      print(e[c])
      im_e.pop(e[c])
      tx_e.pop(e[c])
    c=c+1

lllp


In [ ]:
len(im_e)

24414

In [ ]:
c=0
e=list(im_e.keys())
for i in list(im_e.values()):
    if np.any(np.isnan(i)):
      print(e[c])
    c=c+1


In [ ]:
im_e.pop("lllp")
tx_e.pop("lllp")

In [ ]:
np.any(np.isnan(im_e["dapper"] ))

False

In [ ]:
from random import seed, sample

seed(42)

test_validation_keys = sample(im_e.keys(),800) #sample(image_embeddings.keys(),k=2000)#sample(d.keys(),k=2000)  #this is for sentence ==>  sample(d.keys(),k=2000)

############# check this based on thte dataset and construction of the i_e


validation_keys = test_validation_keys[:400]
test_keys = test_validation_keys[400:]

test_keys = set(test_keys)
validation_keys = set(validation_keys)

#print(list(test_embeddings.keys())[:10])
#print(choices)

test_image = dict()
test_text = dict()

validation_image = dict()
validation_text = dict()

train_image = dict()
train_text = dict()


for item in im_e.keys():
    if item in test_keys: # and item not in no:
        test_image[item] = im_e[item] #d[item]
        test_text[item] =   [tx_e[item]]#text_embedding[item]#t_e[item] #z1[item]
    elif item in validation_keys :# and item not in no:
        validation_image[item] = im_e[item] #d[item]
        validation_text[item] =  [tx_e[item]]#text_embedding[item]#t_e[item] #z1[item]
    else:
      #if item not in no:
        train_image[item] =  im_e[item]
        train_text[item] = [tx_e[item],]#text_embedding[item]# t_e[item]

# Evaluation Setup

In [ ]:
import itertools
def image_to_text_recall_at_k(image_embeddings, image_labels, text_embeddings, text_labels, k=[1, 2, 5, 10], metric='euclidean', verbose = False):
    """
    args:
        embeddings : mini_batch embedding array
                     size = [nof_samples x feature_length]
        labels     : labels corresponding to embeddings
                     size: [nof_samples,]
        k          : recall/precision@k
        metric     : distance metric to evaluate
                     available choices: cosine, euclidean, l1, l2, manhattan, mahalanobis,hamming

    Measure the recall score at k:
    Each test image (query) first retrieves K nearest neighbors from the test set
    and receives score 1 if an image of the same class is retrieved among the K
    nearest neighbors and 0 otherwise. Recall@K averages this score over all the images.

    Measure precision at k:
    Each test image (query) first retrieves K nearest neighbors from the test set
    precision_at_k is average number of correct labels in k neighborhood
    for each image in test set
    """
#    indices = np.transpose(np.argsort(cdist(embeddings, embeddings, metric='euclidean'), axis=0)[:n,:])

    # Find the nearest k[-1]+1 neighbors of each image embedding (the neighbors are text embeddings)
    nbrs = NearestNeighbors(n_neighbors=k[-1], algorithm='auto', metric= metric, n_jobs=-1).fit(text_embeddings)
    distances, indices = nbrs.kneighbors(image_embeddings)

    if verbose:
        print(indices[:25])

    #print(indices)

    #print(distances)

    neighbors_array = [text_labels[i] for i in list(itertools.chain(*indices))]
    neighbors_array = np.reshape(np.asarray(neighbors_array, dtype=np.int32), indices.shape)

    #for i in neighbors_array:
        #print(i)

    #search_array = [image_labels[i] for i in range(image_labels.shape[1])]
    #query_array = np.repeat(np.expand_dims(neighbors_array[:, 0], 1), k[-1]+1, axis=1)
    sparse_array = np.equal(neighbors_array, image_labels).astype(np.float32)

    #print(sparse_array)
    #for i in query_array:
        #print(i)

    recall_at_k = np.zeros((len(k)), )
    prec_at_k = np.zeros((len(k)), )

    for i, n in enumerate(k):
        recall_at_k[i] = np.mean(np.sum(sparse_array[:, :n], axis=1).astype(np.bool).astype(np.float32), axis=0)
        prec_at_k[i] = np.mean(np.mean(sparse_array[:, :n], axis=1), axis=0)

    return prec_at_k, recall_at_k

In [ ]:
def evaluate_image_to_text(test_images, test_texts, basic_model, verbose = False):
    #Convert image names into ints
    label_dict = dict()

    count=0
    array_count = 0
    image_embeddings = np.empty([len(test_images.keys()) ,512])
    image_labels = np.empty([len(test_images.keys()),1])

    num_of_texts =len(test_text.values())#sum([len(emb)for emb in test_texts.values()]) #len(test_texts) *10 #sum([len(emb)for emb in test_texts.values()])
    #print(num_of_texts)
    text_embeddings = np.empty([num_of_texts,512])
    text_labels = np.empty([num_of_texts,1])


    for name in tqdm(test_images.keys()):
          if name not in label_dict.keys():
            label_dict[name] = count
            count += 1
          #print(type(test_images[name]))
          #print(type(test_texts[name]))
          #print(name)
          image_embeddings[array_count,:] = basic_model.predict([np.array(test_images[name]).reshape(1,4096),test_texts[name][0].reshape(1,512)])[0]
          image_labels[array_count]=label_dict[name]
          array_count += 1

    array_count = 0
    for name in tqdm(test_texts.keys()):
      for embedding in test_texts[name]:
          #print(embedding)
          prediction = basic_model.predict([np.array(test_images[name]).reshape(1,4096),embedding.reshape(1,512)])
          #print(prediction)
          text_embeddings[array_count,:] = prediction[1]
          text_labels[array_count] = label_dict[name]
          array_count +=1

    if verbose:
        for i,key in enumerate(test_images.keys()):
            print("########",key,"########",image_labels[i])

    p,r = image_to_text_recall_at_k(image_embeddings, image_labels, text_embeddings, text_labels, k=[1, 2, 5, 10], metric='euclidean',verbose = verbose)
    #print(image_embeddings.shape, image_labels[:10,], text_embeddings[:10,], text_labels[:10,])

    return p,r


In [ ]:
np.set_printoptions(threshold=np.inf)
p,r = evaluate_image_to_text(test_image, test_text, basic_model)
print(p,r)

#p,r = evaluate_image_to_text(validation_image, validation_text, basic_model)
#print(p,r)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/400 [00:00<?, ?it/s]

[0.      0.00125 0.002   0.00175] [0.     0.0025 0.01   0.0175]


# Training

In [ ]:
#Initialize Training Parameters
batch_size = 64
precision = list()
recall = list()
losses = list()
epochs = list()
precision_validation = list()
recall_validation = list()

In [ ]:

################   using obj det OBJS AND AVG VECTORS IMAGS AND PCA


for i in tqdm(range(3000)):
    _,positive_images_minibatch, positive_texts_minibatch, negative_images_minibatch, negative_texts_minibatch = triplet_generator(
        train_image,train_text,0.1,basic_model,batch_size = 64, option=0)
    minibatch_shape = positive_images_minibatch.shape

    loss = merged_model.train_on_batch(
        [positive_images_minibatch, positive_texts_minibatch, negative_images_minibatch, negative_texts_minibatch],
        np.ones(minibatch_shape[0]))
    losses.append(loss)
    if len(epochs)>0:
        epochs.append(epochs[-1] + 1)
    else:
        epochs.append(1)

    if i % 500 == 0:
        p,r = evaluate_image_to_text(test_image, test_text, basic_model)
        p_validation,r_validation = evaluate_image_to_text(validation_image, validation_text, basic_model)
        print('Precision')
        print(p)
        print('Recall')
        print(r)
        print('Validation_Precision')
        print(p_validation)
        print('Validation_Recall')
        print(r_validation)
        print('Loss')
        print(loss)
        precision.append(p)
        recall.append(r)
        precision_validation.append(p_validation)
        recall_validation.append(r_validation)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/3000 [00:00<?, ?it/s]

triplet_loss =  Tensor("model_1/lambda_1/Sum:0", shape=(), dtype=float32)
triplet_loss =  Tensor("model_1/lambda_1/Sum:0", shape=(), dtype=float32)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/400 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/400 [00:00<?, ?it/s]

Precision
[0.145   0.11    0.0645  0.04075]
Recall
[0.145      0.22       0.32249999 0.4075    ]
Validation_Precision
[0.13500001 0.09625    0.061      0.03875   ]
Validation_Recall
[0.13500001 0.1925     0.30500001 0.38749999]
Loss
6.395847320556641


In [ ]:
%cd /content/drive/MyDrive/image_captioning/image_captioning_PyTorch

/content/drive/.shortcut-targets-by-id/17bRhNcxFhDkAannAUkxXcVPO6upXlMF-/image_captioning/image_captioning_PyTorch


In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_Combined_F_VGenemo_MOD_coco_SVD_VGG19_OBJ_COLLNorm_ALPHA_SVD_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_Combined_F_VGenemo_MOD_coco_GCON_i2t_SVD_VGG19_OBJ_COLLNorm_ALPHA_SVD_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_Combined_F_VGenemo_coco_GCON_i2t_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_Combined_F_VGenemo_coco_GCON_i2t_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_Combined_F_VGenemo_coco_i2t_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_Combined_F_VGenemo_coco_i2t_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_VGenemo_PCA_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_VGenemo_PCA_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_VGenemo_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_VGenemo_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_VGenemo_VGG19_OBJ_COLL_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_VGenemo_VGG19_OBJ_COLL_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_Combined_F_VGenemo_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_Combined_F_VGenemo_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
###### WORDS

from tensorflow.keras.models import save_model
#!mkdir models

save_model(
    basic_model,
    'models/basic_model_Combined_F_VGenemo_coc_mod_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/merged_model_Combined_F_VGenemo_coc_mod_SVD_VGG19_OBJ_COLLNorm_trained.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/mscoco/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
import pandas as pd
import altair as alt
#May make notebook large
alt.data_transformers.enable('default', max_rows=None)

df = pd.DataFrame({'Epoch':epochs,'Loss':losses})

df['Rolling'] = df['Loss'].rolling(250, center=True).mean()

orig = alt.Chart(df).mark_line().encode(
    x='Epoch',
    y='Loss',
).interactive()

mean = alt.Chart(df).mark_line(color='red').encode(
    x='Epoch',
    y='Rolling',
).interactive()

orig + mean

alt.LayerChart(...)

In [ ]:
print(precision)

[array([0.    , 0.001 , 0.0018, 0.0025]), array([0.09  , 0.072 , 0.0536, 0.0375]), array([0.124     , 0.0995    , 0.06440001, 0.0454    ]), array([0.133     , 0.0945    , 0.07080001, 0.0478    ]), array([0.133     , 0.1155    , 0.0748    , 0.04940001]), array([0.148 , 0.121 , 0.0794, 0.0515]), array([0.164     , 0.12549999, 0.08320001, 0.0531    ]), array([0.155 , 0.132 , 0.0822, 0.0533]), array([0.16599999, 0.13      , 0.0856    , 0.05560001]), array([0.156     , 0.1285    , 0.0842    , 0.05560001])]


In [ ]:
print(recall)

[array([0.   , 0.002, 0.009, 0.025]), array([0.09      , 0.14399999, 0.26800001, 0.375     ]), array([0.124, 0.199, 0.322, 0.454]), array([0.133     , 0.189     , 0.354     , 0.47799999]), array([0.133     , 0.23100001, 0.37400001, 0.49399999]), array([0.148     , 0.242     , 0.39700001, 0.51499999]), array([0.164     , 0.25099999, 0.41600001, 0.53100002]), array([0.155     , 0.264     , 0.41100001, 0.53299999]), array([0.16599999, 0.25999999, 0.428     , 0.55599999]), array([0.156     , 0.257     , 0.421     , 0.55599999])]


# ORTHOGONAL

In [ ]:
basic_model.load_weights("/content/drive/MyDrive/image_captioning/image_captioning_PyTorch/models/basic_model_Combined_F_VGenemo_coco_SVD_VGG19_OBJ_COLLNorm_ALPHA_SVD_trained.hdf5")



In [ ]:
i_e=im_e
t_e=tx_e
train_image=i_e
train_text=t_e

In [ ]:
j_e={}
d=list(i_e.keys())
c=0
for i in d:
  print(c)
  c=c+1
  j_e[i]= basic_model.predict([i_e[i].reshape(1,4096),np.ones((512,)).reshape(1,512)])[0]


Streaming output truncated to the last 5000 lines.
12158
12159
12160
12161
12162
12163
12164
12165
12166
12167
12168
12169
12170
12171
12172
12173
12174
12175
12176
12177
12178
12179
12180
12181
12182
12183
12184
12185
12186
12187
12188
12189
12190
12191
12192
12193
12194
12195
12196
12197
12198
12199
12200
12201
12202
12203
12204
12205
12206
12207
12208
12209
12210
12211
12212
12213
12214
12215
12216
12217
12218
12219
12220
12221
12222
12223
12224
12225
12226
12227
12228
12229
12230
12231
12232
12233
12234
12235
12236
12237
12238
12239
12240
12241
12242
12243
12244
12245
12246
12247
12248
12249
12250
12251
12252
12253
12254
12255
12256
12257
12258
12259
12260
12261
12262
12263
12264
12265
12266
12267
12268
12269
12270
12271
12272
12273
12274
12275
12276
12277
12278
12279
12280
12281
12282
12283
12284
12285
12286
12287
12288
12289
12290
12291
12292
12293
12294
12295
12296
12297
12298
12299
12300
12301
12302
12303
12304
12305
12306
12307
12308
12309
12310
12311
12312
12313
12314
12315
1

In [ ]:
### x-je, y-te
X=np.array([i[0] for i in list(j_e.values())])
Y=np.array([i for i in list(t_e.values())])

In [ ]:
from scipy import sparse
from scipy.sparse import linalg


In [ ]:
fn=np.dot(np.transpose(X),Y)

In [ ]:
embedding_size = 511
uu, ss, vv = linalg.svds(sparse.csr_matrix(fn), embedding_size)


In [ ]:
jj_e=np.dot(X,uu)

In [ ]:
tt_e=np.dot(Y,np.transpose(vv))

In [ ]:
im_n=list(t_e.keys())
image={}
text={}
for i in range(len(im_n)):
  image[im_n[i]]=jj_e[i]
  text[im_n[i]]=tt_e[i]

In [ ]:
## save JE ,TE
with open("/content/im.pkl","wb") as f:
  pickle.dump(image,f)
## save JE ,TE
with open("/content/tx.pkl","wb") as f:
  pickle.dump(text,f)

In [ ]:
# using the svd 512  +JE
im_n=list(t_e.keys())
emb=[]
z=tt_e
z1=jj_e
for i in range(0,len(z)):
  p=(6*z[i]+2*z1[i])/8
  emb.append(p)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto', metric="euclidean", n_jobs=-1).fit(emb)

#generate embeddings

In [ ]:
from keras.layers.merge import concatenate

i_e={}

for i in range(0,len(test_im)):
  print(i)
  name=list(test_im.keys())[i]
  prediction = basic_model.predict([test_im[name].reshape(1,4096),np.ones((512,)).reshape(1,512)])
  ff=np.dot(prediction[0],uu)
  i_e[name]=ff
